In [2]:
from collections import Counter
import re
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from scipy.stats import entropy as kl_div
from math import log
import requests
from tqdm import tqdm

In [5]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': '172.18.0.3', 'port': 9200}])
res = requests.get('http://172.18.0.3:9200')
print(res.content)

b'{\n  "name" : "bO9m5sh",\n  "cluster_name" : "elasticsearch",\n  "cluster_uuid" : "igPwZ1szS9K6CJ0OeeWr7Q",\n  "version" : {\n    "number" : "6.2.1",\n    "build_hash" : "7299dc3",\n    "build_date" : "2018-02-07T19:34:26.990113Z",\n    "build_snapshot" : false,\n    "lucene_version" : "7.2.1",\n    "minimum_wire_compatibility_version" : "5.6.0",\n    "minimum_index_compatibility_version" : "5.0.0"\n  },\n  "tagline" : "You Know, for Search"\n}\n'


In [4]:
news_train = fetch_20newsgroups(
    data_home='../data/20newsgroups/',
    subset='train', remove=('headers', 'footers', 'quotes'),
    shuffle=True, random_state=666)

In [12]:
for index, data in tqdm(enumerate(news_train.data)):
    doc = {'doc_id': index,
           'doc_text': data}
    try:
        es.index(index='20ng', doc_type='doc', body=doc)
    except RequestError:
        pass

11314it [01:28, 127.29it/s]


In [17]:
import os
from bs4 import BeautifulSoup

path = '../../data/DUC2001/'
gold_path = path + "Summaries"
docs = {}

doc_files = os.listdir(path)
gold_files = os.listdir(gold_path)

for gold_file in gold_files:
    file_name =  gold_file.split('.')[0].upper()
    if file_name not in doc_files:
        print('no file for %s' % gold_file)
        continue

    sum_path = gold_path + '/' + gold_file

    with open(sum_path) as sum_file:                
        summary, doc = sum_file.read().strip().split('Introduction:')
        summary = summary.split('Abstract:')[1]

        docs[file_name] = [doc.strip(), summary.strip()]

for key, val in tqdm(docs.items()):
    es_inp = {
        'doc_id' : key,
        'doc_text' : val[0],
        'gold_summary' : val[1]
    }
    es.index(index='duc', doc_type='doc', body=es_inp)


no file for ap900928-0054.txt
no file for ap890325-0143.txt
no file for ap900322-0200_system.txt


100%|██████████| 301/301 [00:01<00:00, 209.13it/s]
